# Slate Star Codex Reader Survey 2018

This is the data wrangling notebook to accompany my analysis. It's probably not of interest to you unless a) you're a novice data scientist and want to learn some new stuff, or b) you want to see my methodology to understand the strengths and weaknesses of my approach.

TL;DR:

- I dummify the multiple choice questions. This means I convert them into multiple yes/no (1/0) questions so we can mathematically analyze them.

- For questions that allowed an "Other" response, I removed responses that appeared fewer than 3 times, and dummified the rest.

- I removed responses that used strings of letters when numbers were expected. For instance, writing "ten" instead of "10." This happened often enough that's it's not worth the effort to try fixing them all. We just need a stricter survey next time!

- I use machine learning to try predicting missing values. This is largely successful.

**If you enjoy this, let's connect on LinkedIn:**

**https://www.linkedin.com/in/vincefavilla/**

I'm a psychologist and machine learning engineer. Also, I'm in the SF Bay Area and looking for a job outside of academia. :)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('ssc2018public.xlsx')

In [3]:
# Open-ended categoricals; people could enter their own text as "Other" (bad for data analysis!)
# We'll limit them to the most common responses.

to_simplify = '''Country
State
ReligiousDenomination
ReligiousBackground
PoliticalAffiliation
PoliticalChange
PoliticalChangeSSC
Favoriteblog
SSCBenefit
SSCChangeMind
PoliticalDisagreementI
PoliticalDisagreementII'''.split('\n')

# My rule of thumb is "3 is a pattern." So let's remove responses that appear 
# fewer than 3 times and replace them with "Other"
for i in to_simplify:
    value_counts = data[i].str.lower().value_counts()
    value_counts = value_counts[value_counts > 2]
    data[i] = np.where(data[i].str.lower().isin(value_counts.index), data[i].str.title(), 'Other')

In [4]:
# Good categoricals -- need no additional processing
to_dummify = '''Race
Sex
Gender
SexualOrientation
WorkStatus
Profession
Degree
ReligiousViews
ReligiousChange
ReligiousChangeII
MoralViews
LengthofTime
AmericanParties
PoliticalDisagreementIII
TheSystem
IncomeStatus
IncomeChildhood
Cryptocurrency
GWWC
CharityCareer
Depression
Anxiety
OCD
Eatingdisorder
Alcoholism
Drugaddiction
Borderline
Bipolar
Autism
ADHD
Schizophrenia
Adderall
Vegetarian
BirthdateEnding
Harassment1
Harassment2
Harassment3
Harassment4
NaziCategory
MarriageCategory
Handedness
BreatheThrough
Placebo
PreviousSurveys
Mask1
Mask2
Mask3
ParenthesesPalindrome
MapRiddle
SurgeonRiddle
Navon1
Navon2
Sundown
Dancer
Dancer2
SquaresCircles
Tables
Cookies
Reversal
LOC1
LOC2
LOC3
CRTM
Wason
Referrals
PostsRead
Comment
Subreddit
Discord
HiddenOpenThreads
Meetup
PatreonI'''.split('\n')

# Begin dummification!
for i in to_dummify:
    dums = pd.get_dummies(data[i], prefix=i)
    data = pd.concat([data,dums], axis=1)

# We've now simplified these, sot they're ready to dummify too.
for i in to_simplify:
    dums = pd.get_dummies(data[i], prefix=i)
    data = pd.concat([data,dums], axis=1)

In [5]:
# I'm going to attempt to turn these questions into numerical scales.
# It's not perfect, but I think it's good enough to justify doing.

scales = '''Asexuality
Relationshipstyle
EducationComplete
LWID
EAID
SJID'''.split('\n')

relationship = np.zeros_like(data['Relationshipstyle'])
relationship = np.where(data['Relationshipstyle'] == 'Prefer monogamous', 1, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Prefer polyamorous', 0, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Other', 0.5, relationship)
relationship = np.where(data['Relationshipstyle'] == 'Uncertain / no preference', 0.5, relationship)
data['Monogamy'] = pd.Series(relationship)

data['EducationComplete'] = np.where(data['EducationComplete'] == 'Yes', 1, 0)

def yes_no_sorta(col):
    new_col = np.zeros_like(data[col])
    new_col = np.where(data[col] == 'Yes', 1, new_col)
    new_col = np.where(data[col] == 'No', 0, new_col)
    new_col = np.where(data[col] == 'Sorta', 0.5, new_col)
    return pd.Series(new_col)

data['LWID'] = yes_no_sorta('LWID')
data['EAID'] = yes_no_sorta('EAID')
data['SJID'] = yes_no_sorta('SJID')
data['Asexuality'] = yes_no_sorta('Asexuality')

In [6]:
data['Relationship'] = np.where(data['RelationshipStatus'] == 'Single', 0, 1)
del data['RelationshipStatus']

# I'll make a wild assumption that among people with
# 4+ children, the average is 4.5.
data['Children'] = data.Children.str.replace(r'4+', '4.5')

We're going to convert the following columns into numeric values, removing any response that contains non-numeric characters. We're going to lose information the process: imagine someone responding "about 1400" vs. "1400" for their SAT score; the former is now considered an invalid answer. Due to their sheer amount of data, we have few other options. The survey can be improved next time by forcing numeric responses and instructing people to estimate when necessary.

In [7]:
data['IQ'] = pd.to_numeric(data['IQ'], errors='coerce')
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')
data['SATscoresoutof1600'] = pd.to_numeric(data['SATscoresoutof1600'], errors='coerce')
data['SATscoresoutof2400'] = pd.to_numeric(data['SATscoresoutof2400'], errors='coerce')
data['Income'] = pd.to_numeric(data['Income'], errors='coerce')
data['Charity'] = pd.to_numeric(data['Charity'], errors='coerce')
data['OlderBrothers'] = pd.to_numeric(data['OlderBrothers'], errors='coerce')
data['YoungerBrothers'] = pd.to_numeric(data['YoungerBrothers'], errors='coerce')
data['YoungerSisters'] = pd.to_numeric(data['YoungerSisters'], errors='coerce')
data['CRT1'] = pd.to_numeric(data['CRT1'], errors='coerce')
data['CRT2'] = pd.to_numeric(data['CRT2'], errors='coerce')
data['CRT3'] = pd.to_numeric(data['CRT3'], errors='coerce')
data['AutismSpectrumTest'] = pd.to_numeric(data['AutismSpectrumTest'], errors='coerce')
data['GenderRoleTestM'] = pd.to_numeric(data['GenderRoleTestM'], errors='coerce')
data['GenderRoleTestF'] = pd.to_numeric(data['GenderRoleTestF'], errors='coerce')
data['Gender2M'] = pd.to_numeric(data['Gender2M'], errors='coerce')
data['Gender2F'] = pd.to_numeric(data['Gender2F'], errors='coerce')
data['Children'] = pd.to_numeric(data['Children'], errors='coerce')

for i in data.columns:
    if 'BigFive' in i:
        data[i] = pd.to_numeric(data[i], errors='coerce')
        data[i] = np.where(data[i] > 100, data[i].mean(), data[i])
        data[i] = np.where(data[i] < 0, data[i].mean(), data[i])

# Some IQ scores seem a liiittle implausible. Let's replace them with the median.
data['IQ'] = np.where(data.IQ > 175, data.IQ.median(), data.IQ)
data['IQ'] = np.where(data.IQ < 70, data.IQ.median(), data.IQ)

# This question was reversed, so let's fix it
data['GlobalWarming'] = 6 - data['GlobalWarming']

# Combine these two
data['State_Washington Dc'] = data['State_Washington Dc'] + data['State_Washington, Dc']
del data['State_Washington, Dc']

In [8]:
# I'll be ignoring the following features, either because
# they're too open-ended, cause errors, or are too specific to SSC

to_delete = '''Adderall4
BlogReferrals
PostReferrals
PatreonII
PatreonIII
PatreonIV
Classified
PoliticalChangeDescription
SurveyTime
Adderall2
Adderall3
Meetup2
Favoriteblog
Favoritepost'''.split('\n')

for i in to_delete:
    del data[i]

I don't always like missing data, so we're going to use some machine learning to infer the missing values. I'll make datasets available both with and without this technique, so you can use whichever version you prefer.

In [9]:
data = data.select_dtypes(include=[np.number])
data = data.astype(float)
data.columns = data.columns.str.replace('[', '(')
data.columns = data.columns.str.replace(']', ')')
data.columns = data.columns.str.replace('<', 'lessthan')
data.to_csv('ssc2018public_cleaned.csv')

# Number of missing values per question
missing = pd.DataFrame(data.isnull().sum().sort_values(ascending=False))
missing[missing > 0].dropna()

,0
SATscoresoutof2400,5815.0
ChildrenHappiness,5472.0
IQ,5268.0
SATscoresoutof1600,4967.0
AnxietyScale,4426.0
AnxietyFIXED,3256.0
BigFiveE,2960.0
BigFiveC,2953.0
BigFiveA,2953.0
BigFiveO,2952.0


In [10]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Get null columns
null_cols = data.isnull().sum()
null_cols = list(null_cols[null_cols != 0].index)

for i in null_cols:
    x = data.fillna(data.median())
    y = x.pop(i)
    
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2)
    reg = XGBRegressor()
    reg.fit(xtrain, ytrain)      
    pred = reg.predict(xtest)
    r2 = r2_score(ytest, pred)
    
    # If we can reasonably predict these values, do so
    if r2 > .20:
        print('regressing', i, 'with an r2 of', round(r2, 3))
        data['predicted'] = reg.predict(data.fillna(data.median()).drop([i], axis=1))
        data[i] = np.where(data[i].isnull(), data['predicted'], data[i])
        del data['predicted']
    
    # Otherwise, just take the median
    else:
        print('averaging', i)
        data[i] = data[i].fillna(data[i].median())

regressing Age with an r2 of 0.664
regressing GenderConformity with an r2 of 0.391
regressing Children with an r2 of 0.449
averaging ChildrenHappiness
regressing GenderThoughts with an r2 of 0.225
averaging IQ
averaging SATscoresoutof1600
averaging SATscoresoutof2400
regressing Percentile with an r2 of 0.201
regressing SSCFavorability with an r2 of 0.349
regressing SSCAgreement with an r2 of 0.335
regressing Rationalistfavorability with an r2 of 0.293
regressing PoliticalSpectrum with an r2 of 0.729
averaging PoliticalInterest
regressing GlobalWarming with an r2 of 0.529
regressing Immigration with an r2 of 0.446
regressing MinimumWage with an r2 of 0.537
regressing GayMarriage with an r2 of 0.519
regressing Feminism with an r2 of 0.505
regressing HumanBiodiversity with an r2 of 0.303
regressing BasicIncome with an r2 of 0.362
regressing DonaldTrump with an r2 of 0.574
regressing PaulRyan with an r2 of 0.402
regressing BarackObama with an r2 of 0.49
regressing BernieSanders with an r2 

In [11]:
data.to_csv('ssc2018public_inferred.csv')